# Exploratory Data Analysis (EDA) for Food Reviews Dataset

## Overview of the Datasets


### Load the datasets

In [1]:
# Import required libraries
import pandas as pd
from pandas.core.arrays.categorical import contains

In [2]:
# Load the data from the CSV and TXT files
bad_lines = []
def collect_bad_line(line):
    bad_lines.append(line)
    return None  # skip

nutrition_df = pd.read_csv('data/Nutrition.csv', sep=";")
restaurants_df = pd.read_csv('data/Restaurants.csv', sep=";")
reviews_df = pd.read_csv('data/Reviews.txt', sep="\t")
recipes_df = pd.read_csv("data/Recipes.csv", sep=";", engine="python", on_bad_lines=collect_bad_line)

print("loaded:", recipes_df.shape)
print("bad lines captured:", len(bad_lines))
print("example bad line:", bad_lines[0][:2000] if bad_lines else "none")


loaded: (402037, 13)
bad lines captured: 0
example bad line: none


### Recipes Dataset

In [3]:
print('====== Recipes Dataset ======')
print('Number of rows: ', recipes_df.shape[0])
print('Number of columns: ', recipes_df.shape[1])
print('Number of Unique Recipe Categories: ', len(list(set(recipes_df['RecipeCategory'].unique()))))
print('Number of duplicates: ', recipes_df.duplicated().sum())
print('Number of Null Values: ', recipes_df.isnull().sum().sum())
print('Number of rows with null values: ', len(recipes_df[recipes_df.isnull().any(axis=1)].index))
print('Columns with Null Values: ', recipes_df.isnull().sum())
print('\n')
recipes_df.head()


====== Recipes Dataset ======
Number of rows:  402037
Number of columns:  13
Number of Unique Recipe Categories:  295
Number of duplicates:  0
Number of Null Values:  500854
Number of rows with null values:  360568
Columns with Null Values:  RecipeId                           0
Name                               0
CookTime                       65291
PrepTime                           0
DatePublished                      0
Images                             1
RecipeCategory                   449
Keywords                       10909
RecipeIngredientQuantities         2
RecipeIngredientParts              0
RecipeServings                145767
RecipeYield                   278435
RecipeInstructions                 0
dtype: int64




,RecipeId,Name,CookTime,PrepTime,DatePublished,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,1999-08-09T21:46:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,PT25M,PT4H,1999-08-29T13:12:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,PT5M,PT30M,1999-09-05T19:52:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,PT20M,PT24H,1999-09-03T14:54:00Z,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,PT30M,PT20M,1999-09-19T06:19:00Z,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.0,NaN,"c(""Mix everything together and bring to a boil..."


### Nutrition Dataset


In [4]:
print('====== Nutrition Dataset ======')
print('Number of rows: ', nutrition_df.shape[0])
print('Number of columns: ', nutrition_df.shape[1])
print('Number of Unique Recipe Categories: ', len(list(set(nutrition_df['RecipeCategory'].unique()))))
print('Number of duplicates: ', nutrition_df.duplicated().sum())
print('Number of Null Values: ', nutrition_df.isnull().sum().sum())
print('Number rows with null values: ', [r for r in nutrition_df[nutrition_df.isnull().any(axis=1)].index])
print('Columns with Null Values: ', nutrition_df.isnull().sum())
print('\n')
nutrition_df.head()

====== Nutrition Dataset ======
Number of rows:  81411
Number of columns:  12
Number of Unique Recipe Categories:  242
Number of duplicates:  3
Number of Null Values:  21
Number rows with null values:  [1369, 3416, 3526, 3645, 4590, 4591, 75670, 76570, 80674, 80701, 80790, 81410]
Columns with Null Values:  Name                   0
Description            5
RecipeCategory         7
Calories               1
FatContent             1
SaturatedFatContent    1
CholesterolContent     1
SodiumContent          1
CarbohydrateContent    1
FiberContent           1
SugarContent           1
ProteinContent         1
dtype: int64




,Name,Description,RecipeCategory,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,Frozen Desserts,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2
1,Biryani,Make and share this Biryani recipe from Food.com.,Chicken Breast,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4
2,Best Lemonade,This is from one of my first Good House Keepi...,Beverages,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3
3,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,Soy/Tofu,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3
4,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,Vegetable,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3


### Restaurants Dataset

In [5]:
print('====== Restaurants Dataset ======')
print('Number of rows: ', restaurants_df.shape[0])
print('Number of columns: ', restaurants_df.shape[1])
print('Number of Unique Countries: ', len(list(set(restaurants_df['Country'].unique()))))
print('Number of duplicates: ', restaurants_df.duplicated().sum())
print('Number of Null Values: ', restaurants_df.isnull().sum().sum())
print('The rows of the null values: ', [r for r in restaurants_df[restaurants_df.isnull().any(axis=1)].index])
print('\n')
restaurants_df.head()

====== Restaurants Dataset ======
Number of rows:  9550
Number of columns:  21
Number of Unique Countries:  15
Number of duplicates:  0
Number of Null Values:  0
The rows of the null values:  []




,Restaurant ID,Restaurant Name,Country,City,Address,Locality Verbose,Longitude,Latitude,Cuisines,Currency,...,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,Nummber of dishes in cuisines,Average cost of two in USD
0,1600219,12212,India,Nashik,"Shop 10, Ramrajya Building 7, Samarth Nagar, N...","College Road, Nashik",73.754636,20.006690,Fast Food,Indian Rupees(Rs.),...,0,0,0,2,3.5,3.70,3,80,1,4.664
1,17057397,'Ohana,USA,Orlando,"1600 Seven Seas Drive, Lake Buena Vista, FL 32830","Disney World Area, Orlando",-81.585226,28.405437,Hawaiian,Dollar($),...,0,0,0,3,4.5,4.65,5,1151,1,45.000
2,18222559,{Niche} - Cafe & Bar,India,New Delhi,"2nd & 3rd Floor, M-16, M Block, Outer Circle, ...","Connaught Place, New Delhi",77.222507,28.631516,"North Indian, Chinese, Italian, Continental",Indian Rupees(Rs.),...,0,0,0,3,4.1,4.25,4,492,4,17.490
3,113702,@Mango,India,Ahmedabad,"Opposite Sindhu Bhawan, Bodakdev, Ahmedabad","Bodakdev, Ahmedabad",72.501764,23.040163,"North Indian, Continental, Mexican, Italian",Indian Rupees(Rs.),...,0,0,0,3,4.1,4.25,4,769,4,9.328
4,3100446,#45,India,Mangalore,"Ground Floor, Trinity Commercial Complex, Near...","Attavar, Mangalore",0.000000,0.000000,Cafe,Indian Rupees(Rs.),...,0,0,0,2,3.6,3.70,3,209,1,6.996


### Reviews Dataset
This is the unstructured dataset that contains the reviews from users. the reviews are supposed to be on a single line, however this is not the case and a single review may be split up into more than one line.

In [6]:
print('====== Reviews Dataset ======')
print('Number of rows: ', reviews_df.shape[0])
print('Number of columns: ', reviews_df.shape[1])
print('Number of Unique Authors: ', len(list(set(reviews_df['AuthorId'].unique()))))
print('Number of duplicates: ', reviews_df.duplicated().sum())
print('Number of Null Values: ', reviews_df.isnull().sum().sum())
print('Number of rows with null values: ', len(reviews_df[reviews_df.isnull().any(axis=1)].index))
print('Columns with Null Values: \n', reviews_df.isnull().sum())
print('\n')
reviews_df.head(20)

====== Reviews Dataset ======
Number of rows:  317099
Number of columns:  7
Number of Unique Authors:  87644
Number of duplicates:  1446
Number of Null Values:  373388
Number of rows with null values:  105379
Columns with Null Values: 
 ReviewId              2
RecipeId          17983
AuthorId          24121
AuthorName        60713
Review            71159
DateSubmitted     94054
DateModified     105356
dtype: int64




,ReviewId,RecipeId,AuthorId,AuthorName,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,9,4523,2046,Gay Gilmore ckpt,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
2,14,44,2085,Tony Small,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
3,21,148,2156,Darlene Blythe,Would someone please check the Nutrition Facts...,2000-06-02T10:01:00Z,2000-06-02T10:01:00Z
4,22,517,2046,Gay Gilmore ckpt,thought this was terrific!,2000-02-25T09:02:00Z,2000-02-25T09:02:00Z
5,23,4684,2046,Gay Gilmore ckpt,this is absolutely delicious. i even served i...,2000-02-25T09:06:00Z,2000-02-25T09:06:00Z
6,25,3431,2046,Gay Gilmore ckpt,leeks on a pizza?! it was really delicious. ...,2000-04-07T11:06:00Z,2000-04-07T11:06:00Z
7,28,1451,2416,jessontanya,Where does the bbq come in,2000-08-01T19:17:30Z,2000-08-01T19:17:30Z
8,33,4053,1986,Kevin Connolly,"""This was a fine sandwich",I'll definitely be making it again. I think y...,NaN
9,"""",2000-08-26T12:35:25Z,2000-08-26T12:35:25Z,NaN,NaN,NaN,NaN


In [7]:
# reviews that have recipe ids that are not present in the recipes dataset
invalid_reviews = reviews_df[reviews_df['RecipeId'].isin(recipes_df['RecipeId'])]
print('Number of reviews with invalid recipe ids: ', invalid_reviews.shape[0])

Number of reviews with invalid recipe ids:  0


## Cleaning the Datasets
With every single dataset we will have to clean it in order to make it suitable for the construction of the knowledge graph. In all datasets duplicated rows are removed. Null values are treated accordingly based on the importance of the attribute. Also, only recipes which have nutritional information are kept, therefore, keeping only recipes that are present in both the Recipes and Nutrition datasets.

#### Relevant Columns in each dataset:
- For the **Recipe dataset**, <u>remove</u> the `RecipeYield`, `RecipeServings`, and `Images` columns as they are not relevant to the task. Replace the null values in the `CookTime` column with the median cook time. Also, <u>remove</u> the rows that have the null values in the `RecipeCategory` and `Keywords` column as they are crucial attributes for the knowledge graph.

- In the **Nutrition dataset**, <u>keep</u> all the columns as they are relevant to the construction of the knowledge graph. But <u>remove</u> the rows that do not have a `RecipeCategory` value and nutritional values.

- In the **Restaurants dataset**, <u>remove</u> the `Is delivering now`, `Switch to order menu`, `Price range`, and `Nummber of dishes in cuisines` columns as they are not relevant to the task.

- In the **Reviews dataset**, which is unstructured, is preprocessed to deal with all the reviews that are split into multiple lines and columns. All columns are <u>kept</u>.

In [8]:
# Remove duplicates from all the datasets
recipes_df.drop_duplicates(inplace=True)
restaurants_df.drop_duplicates(inplace=True)
nutrition_df.drop_duplicates(inplace=True)

# Remove irrelevant columns from the datasets
recipes_df.drop(columns=["RecipeYield", "RecipeServings", "Images"], inplace=True)
restaurants_df.drop(columns=["Is delivering now", "Switch to order menu", "Price range", "Nummber of dishes in cuisines"], inplace=True)

# Remove rows with null values in the RecipeCategory and Keywords columns in the Recipes dataset
recipes_df.dropna(subset=['RecipeCategory', 'Keywords', 'RecipeIngredientQuantities'], inplace=True)
nutrition_df.dropna(inplace=True) # remove rows with null values in the Nutrition dataset

# Remove all the recipes in the Recipes dataset that are not present in the Nutrition dataset
recipes_df = recipes_df[recipes_df['Name'].isin(nutrition_df['Name'])]

In [9]:
# Clean the cook and prep times of the Recipes dataset
def clean_cook_time_column(df, column):
    for index, row in df.iterrows(): # iterate through each row in the dataframe
        minutes = 0 # initialise the minutes variable
        time_str = row[column] # get the cook time string
        if pd.isna(time_str): # if the cook time value is null, skip the row
            df.at[index, column] = str(0)
            continue
        time_str = time_str.replace('PT', '') # remove the 'PT' prefix from the cook time values
        for char in time_str: # iterate through each character in the cook time string
            if char == "H":
                minutes += int(time_str[:time_str.index(char)]) * 60 # add the hours to the minutes variable
                time_str = time_str[time_str.index(char)+1:]
            elif char == "M":
                minutes += int(time_str[:time_str.index(char)]) # add the minutes to the minutes variable
        df.at[index, column] = str(minutes) # replace the cook time string with the minutes variable
    df[column] = pd.to_numeric(df[column], downcast='float') # convert the cook time values to numeric
    df[column].fillna(df[column].median()) # replace null cook time values with median time

# Convert CookTime and PrepTime to numeric and replace null values with median
clean_cook_time_column(recipes_df, 'CookTime')
clean_cook_time_column(recipes_df, 'PrepTime')

#### Clean the Reviews dataset

In [14]:
# String Type Constants
NL = 0
DT = 1
Z = 2
NULL = 3


def is_datetime_string(s):
    if isinstance(s, str) and len(s) == 20 and s[4] == '-' and s[7] == '-':
        return True
    return False

def clean_line_and_split(line):
    # remove the double quotes from the line and split it by tab character
    line = line.replace('"""', "").replace('""', "").replace('"', "")
    split_line = line.split("\t") # split the line based on the tab character
    split_line = [s.strip() for s in split_line] # remove leading and trailing whitespace from each element
    return split_line

def get_string_types_from_split_line(row: list[str]):
    string_types = []
    for i in row:
        if i == '' or i is None or i.lower() == 'nan':
            string_types.append(NULL)
        elif is_datetime_string(i):
            string_types.append(DT)
        elif i.isdigit():
            string_types.append(Z)
        else:
            string_types.append(NL)
    return string_types

# Check if a row is in the correct order of the column types
def check_correct_row_order(row: list[str]):
    st = get_string_types_from_split_line(row)
    if len(st) == len(row) == 7 and st[0] == Z and st[1] == Z and st[2] == Z and st[3] == NL and st[4] == NL and st[5] == DT and st[6] == DT:
        return True
    return False


# Function to make the row from an initial line. It iteratively builds and cleans rows until the correct order is achieved
def make_row(all_lines, line_idx, line_string):
    sl = clean_line_and_split(line_string)
    # print(sl)
    # Merge the reviews that have been split into multiple lines or columns
    row = [None for _ in range(7)] # new row to store the cleaned data
    review = "" # string to store the review text
    date1, date2 = None, None # strings to store the first and second datetime
    for n, col in enumerate(sl):
        if col.isdigit() and n < 3: # the first three columns ReviewID, RecipeID, AuthorID should be numeric values
                row[n] = col
        elif n == 3 and not col.isdigit() and not is_datetime_string(col): # the fourth should be the AuthorName column
            row[n] = col
        else:
            if not is_datetime_string(col):
                review += " " + col
            else:
                if not date1 and col != '':
                    date1 = col
                elif not date2 and date1 and col != '':
                    date2 = col
    st_1 = get_string_types_from_split_line(sl)

    # If the previous row had only natural language, empty values and integers and no datetime strings, then we can merge the next
    # row with the current row depending on whether the next row has only datetime strings or natural language and empty values
    next_row_idx = line_idx + 1
    while date1 is None and date2 is None and next_row_idx < len(all_lines):
        if st_1.count(NL) + st_1.count(NULL) + st_1.count(Z) == len(st_1):
        # if not check_correct_row_order(sl):
        #     print('next row needs to be checked')
            # Get the next line
            next_line = clean_line_and_split(all_lines[next_row_idx])
            st_2 = get_string_types_from_split_line(next_line)
            # If the next row ha sonly natural language, then merge the text into the review column
            if st_2.count(NL) + st_2.count(NULL) == len(st_2):
                review += " ".join(next_line)
            # If the next row has only datetime strings, then merge the datetime into the date columns
            if st_2.count(DT) + st_2.count(NULL) == len(st_2):
                extracted_dates = [i for i in next_line if is_datetime_string(i)]
                date1 = extracted_dates[0]
                date2 = extracted_dates[1] if len(extracted_dates) > 1 else date1
            # If the next row has both natural language and datetime strings
            if st_2.count(NL) + st_2.count(DT) + st_2.count(NULL) == len(st_2) and st_2.count(DT) != 0 and st_2.count(NL) != 0:
                extracted_dates = [i for i in next_line if is_datetime_string(i)]
                date1 = extracted_dates[0]
                date2 = extracted_dates[1] if len(extracted_dates) > 1 else date1
                review += " ".join([i for i in next_line if not is_datetime_string(i) and i != '' and i is not None])
        next_row_idx += 1

    row[4] = review.strip()
    row[5] = date1
    row[6] = date2

    return row


clean_reviews = []
with open('data/Reviews.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()
    # print('First 5 lines of the Reviews.txt file:')
    for idx, line in enumerate(lines):
        if idx == 0: continue # skip the header line
        # print(f'======== Line {idx} =======')
        row = make_row(lines, idx, line)
        if check_correct_row_order(row):
            clean_reviews.append(row)

        # print(st_1)
        # print(row)


clean_reviews = pd.DataFrame(clean_reviews, columns=['ReviewId', 'RecipeId', 'AuthorId', 'AuthorName', 'Review', 'DateSubmitted', 'DateModified'])
print(f'Reduced to {len(clean_reviews)} rows from {reviews_df.shape[0]}')
print('Number of rows: ', clean_reviews.shape[0])
print('Number of columns: ', clean_reviews.shape[1])
print('Number of Unique Authors: ', len(list(set(clean_reviews['AuthorId'].unique()))))
print('Number of duplicates: ', clean_reviews.duplicated().sum())
print('Number of Null Values: ', clean_reviews.isnull().sum().sum())
print('Number of rows with null values: ', len(clean_reviews[reviews_df.isnull().any(axis=1)].index))
print('Columns with Null Values: \n', clean_reviews.isnull().sum())
print('\n')
clean_reviews.head(50)


Reduced to 204998 rows from 317099
Number of rows:  204998
Number of columns:  7
Number of Unique Authors:  35869
Number of duplicates:  0
Number of Null Values:  0
Number of rows with null values:  78277
Columns with Null Values: 
 ReviewId         0
RecipeId         0
AuthorId         0
AuthorName       0
Review           0
DateSubmitted    0
DateModified     0
dtype: int64




C:\Users\denve\AppData\Local\Temp\ipykernel_17740\4048700181.py:119: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('Number of rows with null values: ', len(clean_reviews[reviews_df.isnull().any(axis=1)].index))


,ReviewId,RecipeId,AuthorId,AuthorName,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,9,4523,2046,Gay Gilmore ckpt,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
2,14,44,2085,Tony Small,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
3,21,148,2156,Darlene Blythe,Would someone please check the Nutrition Facts...,2000-06-02T10:01:00Z,2000-06-02T10:01:00Z
4,22,517,2046,Gay Gilmore ckpt,thought this was terrific!,2000-02-25T09:02:00Z,2000-02-25T09:02:00Z
5,23,4684,2046,Gay Gilmore ckpt,this is absolutely delicious. i even served i...,2000-02-25T09:06:00Z,2000-02-25T09:06:00Z
6,25,3431,2046,Gay Gilmore ckpt,leeks on a pizza?! it was really delicious. ...,2000-04-07T11:06:00Z,2000-04-07T11:06:00Z
7,28,1451,2416,jessontanya,Where does the bbq come in,2000-08-01T19:17:30Z,2000-08-01T19:17:30Z
8,33,4053,1986,Kevin Connolly,This was a fine sandwich I'll definitely be ma...,2000-08-26T12:35:25Z,2000-08-26T12:35:25Z
9,36,7478,2585,Jennifer Kurzdorfer,Very simple recipe to make! This is a refreshi...,2000-09-05T13:27:34Z,2000-09-05T13:27:34Z


## Save datasets

In [15]:
recipes_df.to_csv('data/cleaned_recipes.csv', index=False)
nutrition_df.to_csv('data/cleaned_nutrition.csv', index=False)
restaurants_df.to_csv('data/cleaned_restaurants.csv', index=False)
clean_reviews.to_csv('data/cleaned_reviews.csv', index=False)